# 实时物体跟踪 例子 
 
在这个 notebook 中，我们会展示如何使用 JetBot 跟踪对象。我们将使用在 [COCO dataset](http://cocodataset.org) 网站上预先训练好的神经网络，以检测90种不同的常见对象。例如包括了

* 人 (index 0)
* 杯子 (index 47)
 
和许多其他的常见对象。你可以点击 [这里查看](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) 所有的对象。

该模型来自 [TensorFlow 对象检测 API](https://github.com/tensorflow/models/tree/master/research/object_detection)，它还提供自定义训练任务，当训练完成，我们就能使用 Jetson Nano 上的 NVIDIA TensorRT 对其进行优化。

这样做使得这个神经网络非常快，能在 Jetson 上实时执行。但是，这 notebook 并不包含所说的训练与优化步骤。
  
无论如何，让我们开始吧。首先，我们要导入``ObjectDetector``类，它采用我们预先训练过的SSD引擎。

### 计算单个摄像头图像的信号

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

在内部，``ObjectDetector``类使用 TensorRT Python API执行我们提供的引擎。
  
它还负责预处理神经网络的输入，以及分析检测到的对象。

现在这只适用于``jetbot.ssd_tensorrt``package创建的引擎。该软件包具有转换阻最佳的TensorFlow对象模型给TensorRT引擎

接下来，让我们初始化我们的相机。我需要300x300的图像像素作为输入。所以，我们需要设置我们的摄像头。
 
> 在内部，Jetson Nano的图像信号处理是使用Carmera类的GStreamer实现的。这是超级快速的方式，大大地减少了CPU的计算量

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

现在，让我们使用一些摄像头输入执行我们的神经网络。默认情况下``ObjectDetector`` 类生成 ``bgr8``格式。然而，如果你想使用不同的格式作为输入，则你可以覆盖默认的预处理功能。 


In [3]:
model

In [22]:
detections = model(camera.value)

print(detections)

[[{'label': 1, 'confidence': 0.9596109390258789, 'bbox': [0.0071196407079696655, 0.06254974007606506, 0.31206566095352173, 0.9844751358032227]}, {'label': 32, 'confidence': 0.6048657894134521, 'bbox': [0.5587514042854309, 0.018432646989822388, 0.9849312901496887, 0.9875426292419434]}]]


如果摄像头的视野中又任何COCO对象时，它们会存储在``detections``变量中。


### 显示检测对象的名称

我们将使用下面的代码打印出检测到的对象。

In [5]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

Textarea(value="[[{'label': 80, 'confidence': 0.39793840050697327, 'bbox': [0.5121945142745972, 0.324544668197…

你应该看到每个图像中检测到的每个标签，相似度和边界框。但在这个例子只有一个图像。


要打印第一个图像中检测到的第一个对象，我们可以调用一下内容

> 如果未检测到任何对象，则可能会抛出错误

In [6]:
from jetbot import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

Image(value=b'', format='jpeg', height='300', width='300')

In [7]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 80, 'confidence': 0.39793840050697327, 'bbox': [0.5121945142745972, 0.32454466819763184, 0.9830870628356934, 0.7929606437683105]}


### 控制Jetbot去跟踪中心目标

现在我们希望我们的JetBot能跟随指定的对象。 为此，我们将执行以下操作

1. 检测与指定匹配的对象

2. 选择最接近摄像机视野中心的物体，就是 "目标" 对象

3. 将机器人转向目标物体，否则徘徊

4. 如果我们被障碍物阻挡，则左转  
      
我们也会创建一些部件，例如用于显示目标对象的标签，JetBot当前的速度和转弯提示。它将根据目标物体之间的距离，控制JetBot转动的速度和摄像头视野的中心。

首先，让我们加载我们的碰撞检测模型。 
为了方便，预先训练的模型也存储在此目录中，但如果你已经训练过避障模型，那你也可以使用该模型。

In [8]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

太好了，现在初始化JetBot，这样我们就可以控制JetBot的电机了。

In [9]:
from jetbot import Robot

robot = Robot()

最后，让我们显示所有的控件在网络上执行代码更新摄像头。

In [35]:
from jetbot import bgr8_to_jpeg
import numpy as np
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """计算目标的xy中心坐标"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """计算2D向量长度"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """找出离图像中心最近的目标"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def dist(v0, v1):
    return np.dot(v0, v1) / np.sqrt(np.dot(v0, v0)) / np.sqrt(np.dot(v1, v1))

def execute(change):
    image = change['new']
    
    # 计算所有被检测到的目标
    detections = model(image)
    
#     x0, y0, x1, y1 = 200, 0, 200, 10
    x0, y0, x1, y1 = change['line']
    w, h  = image.shape[0], image.shape[1]
#     print(w, h)
    cv2.line(image, (x0, y0), (x1, y1), (255,0,0), 10)
    x0 /= w
    y0 /= h
    x1 /= w
    y1 /= h
    
    closest_detection = None
    for det in detections[0]:
        if det['label'] == 1:
            continue;
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

        center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
        center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
#         print(center_x, center_y)
        dis =  dist(np.array([x1-x0, y1-y0]), np.array([center_x - x0, center_y - y0]))
        if closest_detection is None:
            closest_detection = det
        else:
            
            xx, yy = detection_center(closest_detection)
            if dis < dist(np.array([x1-x0, y1-y0]), np.array([xx - x0, yy - y0])):
                closest_detection = det
        
    if closest_detection is None:
        label_widget.value = 1
#         print("------")
    else:
        label_widget.value = closest_detection['label']
        
        bbox = closest_detection['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
        
    # 更新图像小部件
    image_widget.value = bgr8_to_jpeg(image)
# execute({'new': camera.value})

调用下面的代码，将执行功能连接到每帧图像更新。

In [37]:
camera.unobserve_all()
camera.observe(execute, names='value')

In [38]:
robot.stop()

真棒！ 如果机器人未被阻挡，您应该看到在检测到的物体周围用蓝色绘制的框。 目标对象（JetBot设置的对象）将以绿色显示。

检测到对象应该朝向目标转向。 如果它被一个物体挡住，那么它就会向左转。

您可以调用下面的代码，断开相机的处理并停止机器人。

In [38]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()